In [2]:
%pip install scikit-learn opencv-python tensorflow pandas numpy mediapipe

import sklearn
import cv2
import tensorflow as tf
import pandas as pd
import numpy as np
import mediapipe as mp

# Printing versions of installed libraries
print("scikit-learn version:", sklearn.__version__)
print("OpenCV version:", cv2.__version__)
print("TensorFlow version:", tf.__version__)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)
print("Mediapipe version:", mp.__version__)

# scikit-learn version: 1.5.1
# OpenCV version: 4.10.0
# TensorFlow version: 2.16.2
# Pandas version: 2.2.2
# NumPy version: 1.26.4
# Mediapipe version: 0.10.14




scikit-learn version: 1.5.1
OpenCV version: 4.10.0
TensorFlow version: 2.16.2
Pandas version: 2.2.2
NumPy version: 1.26.4
Mediapipe version: 0.10.14


In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split



files = ["dislikes_data.csv","likes_data.csv","middle_finger_data.csv","gun_data.csv","all_good_data.csv"]
frames = [ pd.read_csv(f) for f in files ]
data = pd.concat(frames,ignore_index=True)
Y = data["type"]
X = data.drop("type",axis = "columns")
X.info()

# Splitting the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11538 entries, 0 to 11537
Data columns (total 63 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0      11538 non-null  float64
 1   y0      11538 non-null  float64
 2   z0      11538 non-null  float64
 3   x1      11538 non-null  float64
 4   y1      11538 non-null  float64
 5   z1      11538 non-null  float64
 6   x2      11538 non-null  float64
 7   y2      11538 non-null  float64
 8   z2      11538 non-null  float64
 9   x3      11538 non-null  float64
 10  y3      11538 non-null  float64
 11  z3      11538 non-null  float64
 12  x4      11538 non-null  float64
 13  y4      11538 non-null  float64
 14  z4      11538 non-null  float64
 15  x5      11538 non-null  float64
 16  y5      11538 non-null  float64
 17  z5      11538 non-null  float64
 18  x6      11538 non-null  float64
 19  y6      11538 non-null  float64
 20  z6      11538 non-null  float64
 21  x7      11538 non-null  float64
 22

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import f1_score

class F1ScoreCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).argmax(axis=1)
        val_target = self.validation_data[1]
        f1 = f1_score(val_target, val_predict, average='macro')
        print(f' - val_f1_score: {f1:.4f}')

# Define the model
model = Sequential()

# Add input layer (input_shape is (63,))
model.add(Dense(64, activation='sigmoid', input_shape=(63,)))

# Add hidden layers
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))

# Add output layer with 5 units
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

# Print the model summary
model.summary()


# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[F1ScoreCallback(validation_data=(x_test, y_test))])

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Calculate F1 score on test data
y_pred = model.predict(x_test).argmax(axis=1)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"Test F1 Score: {f1}")


c:\Users\hamra\Desktop\hand pose classification\.venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 64)             │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 5)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,789 (26.52 KB)

 Trainable params: 6,789 (26.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step p - loss: 1.6471 - sparse_categorical_accuracy: 0
 - val_f1_score: 0.2014
289/289 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.6436 - sparse_categorical_accuracy: 0.2362 - val_loss: 1.5257 - val_sparse_categorical_accuracy: 0.3505
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - loss: 1.4392 - sparse_categorical_accuracy: 0.3
 - val_f1_score: 0.4825
289/289 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4246 - sparse_categorical_accuracy: 0.4052 - val_loss: 0.9767 - val_sparse_categorical_accuracy: 0.6269
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - loss: 0.8763 - sparse_categorical_accuracy: 0.6
 - val_f1_score: 0.8599
289/289 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8648 - sparse_categorical_accuracy: 0.6592 - val_loss: 0.6076 - val_sparse_categorical_accuracy: 0.8735
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step p - loss: 0.5609 - sparse_categorical_accuracy: 0.8
 - val_f1_score: 0.9023
289/289 ━━━━━━━━━━━━━━━━━━━━ 1s 2

In [9]:
model.save("hand_pose_clc.keras")